# Introduction

In the last years the NBA has changed dramatically. The playstyle is now more oriented toward early three point shot and fast offensive transitions. Many analyst have been wondering if this basketball revolution benefits to the show and helps the team to win the matches. 

The case is of great interest because the NBA (and in general the american sports) is one of the first sports where the data analytics are heavily exploited to increase the efficiency of the teams. The raising of three point shots comes just from the Big Data analysis.

One of the most relevant example is the landing in NBA of Daryl Morey as the General Manager of the Houston Rockets in 2006: his experience in the computer science and statistics world leads to a data driven playstyle. The criticisms were not missing from NBA fans and former players. Nevertheless, data are now used by any NBA team and it is one crucial aspect to take into account to build a winning team.

In our analysis we want to explore the efficiency of the three point shots and asking if there is an ideal amount of threes to maximize the chance of victory. In order to achieve our purposes we build a machine learning model capable to predict the probability to win. Thereafter we adopt our machine learning model to visualize the non-linear dependence between the three point shots attempted and the chance of winning. 

In [ ]:
#Importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from IPython.display import Markdown, display
import random

# definition of the functions

def Single_column_variation(df = None, column = None, min_value = 0, max_value = 40):
    """This function takes in input a dataframe df and duplicate in a list of dataframe. 
    The new dataframes are built from one row of the original dataframe.
    All the column are fixed, and the column selected range from 0 to max_value."""
    
    df_list = list()

    for i in range(len(df)-1):
        
        df_list.append(pd.DataFrame(df.iloc[i]).transpose())           
        
        df_list[i] = pd.DataFrame(np.repeat(df_list[i].values,
                                  max_value-min_value, axis=0), columns=df_list[i].columns)  # Duplicate each row in a new dataframe
    
     
        for k in range(min_value,max_value):
            df_list[i][column][k] = k    # modify the value of the selected column 

    return df_list


def Best_stats(df, model=None):
    """This function predict the statistics that maximize the probability of winning"""
    
    results = list()
    max_prob = list()

    for i in range(len(df)-1):
        
        results.append(model.predict_proba(df[i])[:,1])  #Predict the probability to win for each value of the selected varying column
        max_prob.append(results[i].argmax())   # Estimate the best stats
        
    return results, max_prob

def printmd(string):
    display(Markdown(string))

# Overview of the data

To build our machine learning model we adopt the NBA games dataset available on Kaggle: https://www.kaggle.com/nathanlauga/nba-games

In our model we use 2 dataset:
* games.csv : all games from 2004 season to last update with the date, teams and some details like number of points, etc.
* games_details.csv : details of games dataset, all statistics of players for a given game

From "games.csv" we select the season of each game and merge with the "games_details" dataframe to include in our model the season in which it was played every game. 

From the first approach we immediately see that there are some NaN value, in particular in the "COMMENT" column. Therefore we need to clean our dataset properly before building the model. Here is a brief overlook of our data:

In [ ]:
# Read the data
df = pd.read_csv('../input/nba-games/games_details.csv')
seasons = pd.read_csv('../input/nba-games/games.csv')[["GAME_ID","SEASON"]]

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.drop(["GAME_ID","TEAM_ID","PLAYER_ID"],axis=1).describe().transpose()

# Data cleaning

In this section we will select the data in which we are interested and detect some anomaly in the dataset. At the end of this section the dataset will be clean suited for an exploratory data analisys and the construction of our machine learning model.

### Duplicated data

The set contain 324 duplicated data; some player stats are repeated more than one time. Once we have identified the duplicate, we dropped them.

In [ ]:
df[df.duplicated(subset=["GAME_ID","PLAYER_ID"], keep="first")]

In [ ]:
df.drop_duplicates(subset=["GAME_ID","PLAYER_ID"],keep="first",inplace=True)

### Group the statistics for game

We are interested in the statistics for each NBA game. Therefore, we group and sum the statistics of every player for a fixed game. After that, some statistics like the player_id, the plus minus and the shot percentages lose meaning, so we drop it and, in the case of the shot percentages, we recalculate them. With this process, we lose some NaN values and some columns in which we are not interested.

In [ ]:
df = df.groupby(["GAME_ID","TEAM_ID"]).sum()
df = df.reset_index()
df = df.drop(['PLAYER_ID', 'FG_PCT','FG3_PCT','FT_PCT','PLUS_MINUS'],axis=1)

df["FG_PCT"] = df["FGM"]/df["FGA"]*100
df["FG3_PCT"] = df["FG3M"]/df["FG3A"]*100
df["FT_PCT"] = df["FTM"]/df["FTA"]*100

### Null values and anomalous data

The new columns show one of the anomaly of our dataset. We have a row in which the percentages are null, due to 0 shot attempts in a game. We can easily detect that the game with the ID = 10500109 has three unique rows. Obviously it is impossible that three team are involved in a single match. The suspicious game has one of the row with all zeros; it is clearly an error in the composition of the dataset, therefore we can eliminate from our set.

In [ ]:
print (f"There is still {df.isna().sum().sum()} null values.\n")
print ("There is a game with 3 unique rows.\n")

print(df["GAME_ID"].value_counts().sort_values().tail(5))

display(df[df["GAME_ID"] == 10500109])

#dropping the null row

df = df.drop(index=335,axis=0)
df = df.reset_index().drop("index",axis=1)

In order to introduce the feature "Victory", we run a check of possible "tie game" (the code will be used again in the next section). In indicates some error in the computation of the points, because a tie at the end of the fourth quarter would be resolved in the overtime (the dataset take into account this scenario). This happens because some player stats record are missing. After a manual cross check with the games.csv set, we fixed the error.

In [ ]:
df.loc[7002, "PTS"] = 90
df.loc[7623,"PTS"] = 103 

### Victory label

In order to train our machine learning model to estimate the chance of victory of a team based on some of its statistics, we introduce the "VICTORY" label, calculated checking the points made from the two teams in a fixed game.

In [ ]:
df = df.sort_values("GAME_ID")   #Sort the rows by GAME_ID, it is an extra check to avoid any bug in the next rows of the code
df["VICTORY"] = ""

for i in range(0,len(df)-1,2):
    if df["PTS"][i] > df["PTS"][i+1]:   # Check which of the two team has won and update the "VICTORY" column
        
        df.loc[i , "VICTORY"] = "Yes"
        df.loc[i+1 , "VICTORY"] = "No"
        
    elif df["PTS"][i] < df["PTS"][i+1]:
        
        df.loc[i , "VICTORY"] = "No"
        df.loc[i+1 , "VICTORY"] = "Yes"  
        
    else:                               # Check for eventual problematic rows
        print("Tie game?")
        print(df.iloc[i])

Now we add to the dataframe the season label, in order to explore the evolution of the stars over the last years. At this point the dataset is clean and ready to be explored.

In [ ]:
df = pd.merge(df,seasons,how="left",on="GAME_ID")

In [ ]:
df.head()

In [ ]:
df.info()

# Exploratory Data Analysis

As mentioned earlied, we want to focus on the offensive side of the NBA game, with particular attention to the role of the three point shots in the playstyle of NBA teams. 

Here we have the mean of the three point shots per game attempted from the 2003 to the 2020. The trend is constantly increasing from 2012. We have shifted from 15 threes per game in 2003 to 35 threes attempted per game in 2020. That's correspond to an increase of over 200% in less than 20 years. 

Therefore, the hypothesis of a more and more central role of this aspect of the game is confirmed, but how this influence on the win? The figure shows slight difference between the three attempted for the winning and the losing team.

In [ ]:
plt.figure(figsize=(10,5),dpi=150)
ax = sns.barplot(data=df,x="SEASON",y="FG3A",hue="VICTORY")

ax.set(ylabel="Three point shots attempted", xlabel="Season")
ax.set_title("Three point shots attempted evolution")

plt.xticks(rotation=90);

With the growth of threes attempted, let's see the percentages evolution over the years. As we can see in the figure below, despite the increasing threes attempted, the average of the threes percentages remains the same around 30-35%. This emphasizes the efficiency of the actual NBA player, capable to maintain optimal percentages even if the volume of long distance shots is larger every year.

Furthermore, the figure suggests an interesting result. As opposed to the previous figure, the percentages are a strong indicator of the success of a team: the average pergentages for a losing or a winning team differs by at least 5%. We have the first crucial indication of the offensive aspect of the game: an efficient offense with high percentages increase the performance of a team.  

In [ ]:
plt.figure(figsize=(8,5),dpi=150)
ax = sns.barplot(data=df,x="SEASON",y="FG3_PCT",hue="VICTORY")

ax.set(ylabel="Three point shots percentage", xlabel="Season")
ax.set_title("Three point shots percentages evolution")
plt.legend(bbox_to_anchor=(1.05, 1), borderaxespad=0.)

plt.xticks(rotation=90);

At this point we may ask how are related the increasing shots with the efficiency of the teams. Faster action, rapid offensive transition and a growing number of offensive solutions in the first seconds could affect the percentages?

In the scatterplot below we see how for threes the increase of the attempts reduce the variance in the percentages: with few shots (0-10) a team could achieve any percentages, istead with a large amount of threes (50-60) the percentages are all in a range between 30% to 50%.

We obtain a similar relation if we consider the field goal (two point shots + three point shots), but in this case the variance in the percentages is greater even for larger amount of shots.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,5),dpi=200)

plt.suptitle("More shots lead to fixed percentages")

sns.scatterplot(ax = ax[0],data=df,x="FG3A",y="FG3_PCT",hue="VICTORY",style="VICTORY",alpha=0.8)
ax[0].set(ylabel="Three point shots percentage", xlabel="Three point shots attempted")

sns.scatterplot(ax = ax[1],data=df,x="FGA",y="FG_PCT",hue="VICTORY",style="VICTORY",alpha=0.8) 
ax[1].set(ylabel="Field goal percentage", xlabel="Field goal attempted")

plt.show()

We see that the NBA teams have managed to perform more threes keeping good shot percentages. Let's now compare the amount of shots with the points made during a game. Obviously the points are the key factor that leads to the win, so this could suggests the effectiveness of this new playstyle.

There is a linear relationship between the shot attempted and the points made, either by two or three point shots, but we cannot clearly see if they are correlated with the victory.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,5), dpi=200)

plt.suptitle("More shots lead to more points")

sns.scatterplot(ax = ax[0], data = df, y = "PTS", x = "FG3A", hue = "VICTORY",style="VICTORY",alpha=0.8)
ax[0].set(xlabel="Three point shots attempted", ylabel="Points")

sns.scatterplot(ax = ax[1], data=df, y="PTS", x="FGA", hue="VICTORY",style="VICTORY",alpha=0.8)
ax[1].set(xlabel="Field goal attempted", ylabel="Points")

plt.show()

# Machine Learning model

We decide to build three different machine learning model splitting the data in three different epochs:
1. From 2003 to 2006
2. From 2007 to 2015
3. From 2016 to 2020

We select 2006 and 2017 as turning point in the NBA history. The first is the year during which Daryl Morey became the General Manager of the Houston Rockets. As we already said the impact of Daryl Morey on the playstyle, leading to a datadriven NBA, was huge, influencing all the other teams in the following years. The second  is the year of the consacration of Stephen Curry with the second MVP of the regular season and the astonishing record of its Golden State Warriors with 73 wins in the regular season (it is the most winning regular season ever). This player is one of the greatest three point shooter of all time and has been a great influence on the future generations of players. 

Having three models we can have a better description of three different phases without losing in accuracy. 

We adopt a Random Forest Classifier selecting the following features:
* Field goals attempted per game
* Three point shots attempted per game
* Free throws attempted per game
* Field goals percentage per game
* Three point shots percentage per game
* Free throws percentage per game
* Offensive rebounds per game
* Defensive rebounds per game
* Assists per game
* Steels per game
* Blocks per game
* Turnovers per game
* Fouls per game

We decided to drop the amount of field goal made, three point made and points; nevertheless there three features are the most important to be fed in a machine learning model in order to predict the results of a game, we need that these stats are not constrain the prediction. The reason will be clearer in the next section.

In [ ]:
# Divide the data in three parts

first_split = df[df["SEASON"] < 2007]
second_split = df[(df["SEASON"] >= 2007) & (df["SEASON"] < 2016)]
third_split = df[df["SEASON"] >= 2016]

# ML model for the first chunk of data (2003-2012)

    #Split

X1 = first_split.drop(["SEASON","VICTORY","FGM","FTM","FG3M","PTS","GAME_ID","TEAM_ID","REB"],axis=1)
y1 = first_split["VICTORY"]

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=27)

    #Train

model_first_split = RandomForestClassifier(bootstrap=True)
model_first_split.fit(X1_train,y1_train)

    #Predict

y1_pred = model_first_split.predict(X1_test)

# ML model for the second chunk of data (2012-2016)
    
    #Split
    
X2 = second_split.drop(["SEASON","VICTORY","FGM","FTM","FG3M","PTS","GAME_ID","TEAM_ID","REB"],axis=1)
y2 = second_split["VICTORY"]

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=27)

    #Train

model_second_split = RandomForestClassifier(bootstrap=True)
model_second_split.fit(X2_train,y2_train)

    #Predict

y2_pred = model_second_split.predict(X2_test)

# ML model for the third chunk of data (2016-2020)

    #Split

X3 = third_split.drop(["SEASON","VICTORY","FGM","FTM","FG3M","PTS","GAME_ID","TEAM_ID","REB"],axis=1)
y3 = third_split["VICTORY"]

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.3, random_state=27)

    #Train

model_third_split = RandomForestClassifier(bootstrap=True)
model_third_split.fit(X3_train,y3_train)

    #Predict

y3_pred = model_third_split.predict(X3_test)

In [ ]:
printmd("These are the results for the first model (2003-2006)\n")
print(classification_report(y1_test,y1_pred))

printmd("These are the results for the second model (2007-2015)\n")
print(classification_report(y2_test,y2_pred))

printmd("These are the results for the first model (2016-2020)\n")
print(classification_report(y3_test,y3_pred))

# Partial Derivation Plots

Our models predict the victory of a team with an accuracy of 81%. We are satisfied with this accuracy: the outcome of a match depends also on the performance of the adversary, moreover we are not interested in the prediction of winning for a team, but we want to explore the non-linear dependences between the feaures and the label.

We build Partial Derivation Plots (PDP) to represent the relation between the chance to win and the three point shots attempted; for every row of the test dataset (we don't want leakage from the train dataset) we select a single column (in this case the three point shots attempted per game) varying in a range between 0 and 60 and create a new dataframe keeping all the other features constant. By averaging for all the games, we obtain the PDPs.

If we haven't drop from our model field goal made, three point made and points, varying the threes these stats, that are intrisically interacting with each other, lose any sense. In this way we lose in accuracy in the prediction, but we maintain the sense of the model during the partial derivation.

In [ ]:
first_split_list = Single_column_variation(X1_test,"FG3A",0,60)
results_first_split, max_prob_first_split = Best_stats(first_split_list, model_first_split)

second_split_list = Single_column_variation(X2_test,"FG3A",0,60)
results_second_split, max_prob_second_split = Best_stats(second_split_list, model_second_split)

third_split_list = Single_column_variation(X3_test,"FG3A",0,60)
results_third_split, max_prob_third_split = Best_stats(third_split_list, model_third_split)

The figure below shows the change in the NBA playstyle. For the set between 2003-2006 the relation follows a parabola, with the maximum around 20 threes and a minimum after 40 threes. The set between 2007-2015 shows an almost linear relation with the chance increasing with more threes until the plateau present after 45 threes. The set between 2016-2020 shows that the chance to win are strongly dependent on the threes attempted.

In [ ]:
average1 = np.sum(results_first_split,axis=0)/len(results_first_split)
average2 = np.sum(results_second_split,axis=0)/len(results_second_split)
average3 = np.sum(results_third_split,axis=0)/len(results_third_split)

plt.figure(figsize=(7,5),dpi=150)

plt.plot(range(60),average1)
plt.plot(range(60),average2)
plt.plot(range(60),average3)

plt.legend(["2003-2006","2007-2015","2016-2020"])
plt.xlabel("Three point shot attempted")
plt.ylabel("Chance to win") 
plt.title("Partial Dependence Plot")

plt.show()

# Estimation of the ideal amount of three point shots

Selecting the number of threes that maximize the chance to win, we can obtain their distribution. Following our model we observe that in certain games, attempting three point shots lowers the chance to win, therefore we have a significance amount of game with 0 as the ideal amount of three point shots. 

Moreover, the peak in the distributions goes to large number of attempts during the years, confirming the increasingly importance of this aspect of the game. 

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,5), dpi=150)

sns.histplot(ax = ax[0], x = max_prob_first_split, kde=True, stat="probability", binwidth=3)
ax[0].set(xlabel="Best amount of threes", title="2003-2006")

sns.histplot(ax = ax[1], x = max_prob_second_split, kde=True, stat="probability", binwidth=3)
ax[1].set(xlabel="Best amount of threes", title="2007-2015")

sns.histplot(ax = ax[2], x = max_prob_third_split, kde=True, stat="probability", binwidth=3)
ax[2].set(xlabel="Best amount of threes", title="2016-2020")

plt.tight_layout()
plt.show()

# Which playstyle is the best?

We conclude the analysis with a fun challenge. Despite the range of the statistics isn't differ to much between the three periods, this model isn't 100% affordable. So take this last section lightly. 

In this section we predict the victory or defeat of the teams, but adopting the old model (2003-2006) for the modern team (2016-2020) and viceversa. The games are equally distributed between victories and defeats. We observe that the win rates lower to 38% for the old team playing in the modern NBA and raises to 63% for the modern team playing in the old NBA. 

In [ ]:
old_school_pred = model_third_split.predict_proba(X1)
new_school_pred = model_first_split.predict_proba(X3)

lose_mean_old = old_school_pred.mean(axis=0)[0]
win_mean_old = old_school_pred.mean(axis=0)[1]
lose_mean_new = new_school_pred.mean(axis=0)[0]
win_mean_new = new_school_pred.mean(axis=0)[1]

print("Old style team playing in the modern NBA win rate  =", win_mean_old,
     "\nNew style team playin in the past NBA win rate =", win_mean_new)